In [1]:
import pickle
import numpy as np
from dataset import Dataset
import plotly.graph_objects as go

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier

In [2]:
with open("sensor_data_filtered.pkl", "rb") as f:
    sensor_data = pickle.load(f)

with open("sensor_labels.pkl", "rb") as f:
    labels = pickle.load(f)

with open("interpolation_functions.pkl", "rb") as f:
    interp_funcs = pickle.load(f)

In [ ]:
day = 3
mat = 2
for sensor in range(8):
    print(f"Day {day}, Mat {mat}, Sensor {sensor}")
    dataset = Dataset(sensor_data, labels, interp_funcs)
    X, y = dataset.get_sensor_cls(day, mat, sensor)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.15, random_state=42)

    clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print(f"Dec. Tree: {accuracy:.3f}")

    clf = GaussianNB()
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print(f"GaussianNB: {accuracy:.3f}")

    extra_tree = ExtraTreeClassifier(random_state=0)
    clf = BaggingClassifier(extra_tree, random_state=0)
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print(f"BaggingTree: {accuracy:.3f}")

    clf = SVC(kernel="linear")
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print(f"SVM: {accuracy:.3f}")

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = LogisticRegression(max_iter=2000)
    clf = clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print(f"LogReg: {accuracy:.3f}")

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = MLPClassifier(max_iter=2000)
    clf = clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print(f"MLP: {accuracy:.3f}")

    clf = XGBClassifier(objective="multi:softprob")
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print(f"XGB: {accuracy:.3f}")

    print("-" * 40)

In [3]:
day = 1
dataset = Dataset(sensor_data, labels, interp_funcs)
X, y, time_arr = dataset.get_sensor_cls(day, 1, 0)
print(X.shape, y.shape, time_arr.shape)
X_pair, y_pair, time_arr_pair = dataset.get_sensor_pair_cls(day, 1, (0, 1))
print(X_pair.shape, y_pair.shape, time_arr_pair.shape)

(400, 10) (400,) (400,)
(400, 20) (400,) (400,)


In [9]:
def plot_data(X, time_arr):
    data_T = X.T
    fig = go.Figure()

    for i, data in enumerate(data_T):
        fig.add_trace(go.Scatter(x=time_arr,
                                 y=data,
                                 mode="markers",
                                 name=f"H.S.{f"S1.{i}" if i < 10 else f"S2.{i-10}"}"))

    fig.update_layout(title="Data", title_x=0.5, width=1000, height=800)
    fig.update_traces(marker=dict(size=2))
    fig.update_yaxes(type="log")
    fig.show()

In [11]:
plot_data(X, time_arr)